In [2]:

import pandas as pd

In [6]:
hgtdf=pd.read_csv("hgt_origin_annotations.csv",index_col=0)

In [9]:
import numpy as np
import os
import ast

In [11]:
for n, row in hgtdf.iterrows():
    df=pd.read_csv(f"blast_round_two_data/{n}.tsv",sep="\t")
    gene=";".join(n.split(";")[0:2])
    interv=ast.literal_eval(n.split("_")[-1])
    leng=interv[1]-interv[0]+1
    df["cov"]=(np.array(df.qend.astype(float))-np.array(df.qstart.astype(float))+1)/leng
    dfo=df[df["cov"]>.30]
    dfo=dfo[~dfo.sphylums.astype(str).str.contains("Arthropoda")]
    dfo=dfo[~dfo.sphylums.astype(str).str.contains("Rotifera")]
    dfo=dfo[dfo.staxids.astype(str)!="nan"]
    dfo=dfo[dfo.staxids!=32630]
    dfm=dfo.iloc[0:30000,:]
    dfmeta=dfm[dfm.skingdoms.astype(str).str.contains("Metazoa")]
    dfhgt=dfm[~dfm.skingdoms.astype(str).str.contains("Metazoa")]
    dfhgt["AI"]=np.log10(dfmeta.evalue.min()+1e-200)-np.log10(dfhgt.evalue+1e-200)
    dfmeta["MI"]=np.log10(dfhgt.evalue.min()+1e-200)-np.log10(dfmeta.evalue+1e-200)
    dfmi=dfm.iloc[0:300,:]
    dfmetai=dfmi[dfmi.skingdoms.astype(str).str.contains("Metazoa")]
    dfhgti=dfmi[~dfmi.skingdoms.astype(str).str.contains("Metazoa")]
    hgtdf.loc[n,["sseqid", "stitle", "sphylums", "sscinames", "pident", "evalue", "bitscore","cov","AI"]]=dfhgt.iloc[0,:].loc[["sseqid", "stitle", "sphylums", "sscinames", "pident", "evalue", "bitscore","cov","AI"]]


/n/home11/rkapoor/.conda/envs/jupyter_3.6/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/n/home11/rkapoor/.conda/envs/jupyter_3.6/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/n/home11/rkapoor/.conda/envs/jupyter_3.6/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a

In [12]:
for n, row in hgtdf.iterrows():
    try:
        df=pd.read_csv(f"hmmer_phylogenetics_results/{n}/filtered_taxonomic_annotated_hmmsearch_result.tsv",sep="\t",index_col=0)
        dfo=df
        dfo=dfo[~dfo.phylum.astype(str).str.contains("Arthropoda")]
        dfo=dfo[~dfo.phylum.astype(str).str.contains("Rotifera")]

        dfm=dfo.iloc[0:30000,:]
        dfmeta=dfm[dfm.kingdom.astype(str).str.contains("Metazoa")]
        dfhgt=dfm[~dfm.kingdom.astype(str).str.contains("Metazoa")]
        AI=np.log10(dfmeta["i-Evalue"].min()+1e-200)-np.log10(dfhgt["i-Evalue"].min()+1e-200)
        hgtdf.loc[n,["hmmer_min_evalue","Hmmer_superkingdom","Hmmer_phylum","Hmmer_sci_name"]]=dfhgt.sort_values("i-Evalue").iloc[0,:].loc[["i-Evalue","superkingdom","phylum","sci_name"]].values
        hgtdf.loc[n,"hmmer_AI"]=AI
    except:
        df=pd.read_csv(f"hmmer_phylogenetics_results/{n}/filtered_taxonomic_annotated_hmmsearch_result.tsv",index_col=0)
        dfo=df
        dfo=dfo[~dfo.phylum.astype(str).str.contains("Arthropoda")]
        dfo=dfo[~dfo.phylum.astype(str).str.contains("Rotifera")]

        dfm=dfo.iloc[0:30000,:]
        dfmeta=dfm[dfm.kingdom.astype(str).str.contains("Metazoa")]
        dfhgt=dfm[~dfm.kingdom.astype(str).str.contains("Metazoa")]
        AI=np.log10(dfmeta["i-Evalue"].min()+1e-200)-np.log10(dfhgt["i-Evalue"].min()+1e-200)
        hgtdf.loc[n,["hmmer_min_evalue","Hmmer_superkingdom","Hmmer_phylum","Hmmer_sci_name"]]=dfhgt.sort_values("i-Evalue").iloc[0,:].loc[["i-Evalue","superkingdom","phylum","sci_name"]].values
        hgtdf.loc[n,"hmmer_AI"]=AI
        df.to_csv(f"hmmer_phylogenetics_results/{n}/filtered_taxonomic_annotated_hmmsearch_result.tsv",sep="\t")

In [13]:
symbionts={}
for x,row in hgtdf.iterrows():
  
    df=pd.read_csv(f"hmmer_phylogenetics_results/{x}/filtered_taxonomic_annotated_hmmsearch_result.tsv",sep="\t",index_col=0)
    df=df.fillna(0)
    df=df[df.phylum!="Arthropoda"]
    df=df[df['i-Evalue']<1e-10].sort_values('i-Evalue').iloc[0:200,:]
    bionts=set()
    
    hgtdf.loc[x,"Symbionts (any)"]=str(set(df[df["sci_name"].astype(str).str.contains("symbiont")].sci_name))
    hgtdf.loc[x,"Wolbachia"]=str(set(df[df["sci_name"].astype(str).str.contains("Wolbachia")].sci_name))
    hgtdf.loc[x,"Rickettsia"]=str(set(df[df["sci_name"].astype(str).str.contains("Rickettsia")].sci_name))
    hgtdf.loc[x,"Coxiella"]=str(set(df[df["sci_name"].astype(str).str.contains("Coxiella")].sci_name))
    hgtdf.loc[x,"Buchnera"]=str(set(df[df["sci_name"].astype(str).str.contains("Buchnera")].sci_name))
    hgtdf.loc[x,"Wigglesworthia"]=str(set(df[df["sci_name"].astype(str).str.contains("Wigglesworthia")].sci_name))
    hgtdf.loc[x,"Burkholderia"]=str(set(df[df["sci_name"].astype(str).str.contains("Burkholderia")].sci_name))
    hgtdf.loc[x,"Vibrio"]=str(set(df[df["sci_name"].astype(str).str.contains("Vibrio")].sci_name))
    hgtdf.loc[x,"Spiroplasma"]=str(set(df[df["sci_name"].astype(str).str.contains("Spiroplasma")].sci_name))
    hgtdf.loc[x,"Hamiltonella"]=str(set(df[df["sci_name"].astype(str).str.contains("Hamiltonella")].sci_name))
    hgtdf.loc[x,"Sodalis"]=str(set(df[df["sci_name"].astype(str).str.contains("Sodalis")].sci_name))
    hgtdf.loc[x,"Borrelia"]=str(set(df[df["sci_name"].astype(str).str.contains("Borrelia")].sci_name))
    hgtdf.loc[x,"Entomoplasma"]=str(set(df[df["sci_name"].astype(str).str.contains("Entomoplasma")].sci_name))

In [15]:
df_cluster=pd.read_csv("corrected_cluster_info.tsv",sep="\t")
intervals=[]
for index, row in df_cluster.iterrows():
    intervals.extend(ast.literal_eval(row.HGT_intervals))

In [16]:
bact=hgtdf[(hgtdf["Donor superkingdom(s)"].str.contains("Bacteria"))&(hgtdf.index.isin(intervals))]
dfsymb=pd.DataFrame()
symb=['Symbionts (any)', 'Wolbachia', 'Rickettsia', 'Coxiella', "Sodalis",'Spiroplasma','Buchnera', 'Wigglesworthia', 'Hamiltonella','Vibrio']
ind=set()
for s in symb:
    dfsymb.loc[s,"Number of HGT Intervals"]=bact[bact[s]!="set()"].shape[0]
    ind=ind|set(bact[bact[s]!="set()"].index)
dfsymb.sort_values(by="Number of HGT Intervals",ascending=False).to_csv("bacteria_donor_genra.csv")
n=dfsymb[dfsymb["Number of HGT Intervals"]>0].shape[0]

In [21]:
hgtdf.columns

Index(['Donor superkingdom(s)', 'chimera_organism', 'sseqid', 'stitle',
       'sphylums', 'sscinames', 'pident', 'evalue', 'bitscore', 'cov', 'AI',
       'hmmer_min_evalue', 'Hmmer_superkingdom', 'Hmmer_phylum',
       'Hmmer_sci_name', 'hmmer_AI', 'Symbionts (any)', 'Wolbachia',
       'Rickettsia', 'Coxiella', 'Buchnera', 'Wigglesworthia', 'Burkholderia',
       'Vibrio', 'Spiroplasma', 'Hamiltonella', 'Sodalis', 'Borrelia',
       'Entomoplasma'],
      dtype='object')

In [22]:
hgtdf.columns=['Tree_inferred_donor_superkingdom(s)', 'chimera_organism', 'blast_hit_sseqid',
       'blast_hit_title', 'blast_hit_phylums', 'blast_sci_name', 'blast_hit_pident', 'blast_hit_evalue', 'blast_hit_bitscore',
       'blast_hit_coverage', 'blast_hit_AI', 'hmmer_min_evalue', 'hmmer_superkingdom', 'hmmer_phylum',
       'hmmer_sci_name', 'hmmer_AI', 'hmmer_Symbionts (any)', 'hmmer_Wolbachia',
       'hmmer_Rickettsia', 'hmmer_Coxiella', 'hmmer_Buchnera', 'hmmer_Wigglesworthia', 'hmmer_Burkholderia',
       'hmmer_Vibrio', 'hmmer_Spiroplasma', 'hmmer_Hamiltonella', 'hmmer_Sodalis', 'hmmer_Borrelia',
       'hmmer_Entomoplasma']

In [30]:
hgtdf.loc[:,['Tree_inferred_donor_superkingdom(s)', 'chimera_organism', 'blast_hit_sseqid', 'blast_hit_title', 'blast_hit_phylums',
       'blast_sci_name','blast_hit_evalue', 'blast_hit_pident', 
       'blast_hit_bitscore', 'blast_hit_coverage', 'blast_hit_AI',
       'hmmer_min_evalue', 'hmmer_superkingdom', 'hmmer_phylum',
       'hmmer_sci_name', 'hmmer_AI', 'hmmer_Symbionts (any)',
       'hmmer_Wolbachia', 'hmmer_Rickettsia', 'hmmer_Coxiella',
       'hmmer_Buchnera', 'hmmer_Wigglesworthia', 'hmmer_Burkholderia',
       'hmmer_Vibrio', 'hmmer_Spiroplasma', 'hmmer_Hamiltonella',
       'hmmer_Sodalis', 'hmmer_Borrelia', 'hmmer_Entomoplasma']].to_csv("hgt_interval_data.tsv",sep="\t")

In [32]:
hgtdf.inde

,Tree_inferred_donor_superkingdom(s),chimera_organism,blast_hit_sseqid,blast_hit_title,blast_hit_phylums,blast_sci_name,blast_hit_pident,blast_hit_evalue,blast_hit_bitscore,blast_hit_coverage,...,hmmer_Coxiella,hmmer_Buchnera,hmmer_Wigglesworthia,hmmer_Burkholderia,hmmer_Vibrio,hmmer_Spiroplasma,hmmer_Hamiltonella,hmmer_Sodalis,hmmer_Borrelia,hmmer_Entomoplasma
Tree_inferred_donor_superkingdom(s),,,,,,,,,,,,,,,,,,,,,
"GCF_000002335.3;XP_015833841.1;HGT_(97,339)",Bacteria,Tribolium castaneum (red flour beetle),MVN24565.1,MVN24565.1 tyrosine-type recombinase/integrase...,Bacillota,Staphylococcus aureus,42.3,7.290000e-24,106.0,0.551440,...,set(),set(),set(),set(),set(),set(),set(),set(),set(),set()
"GCF_000002335.3;XP_015834127.1;HGT_(251,516)",Bacteria,Tribolium castaneum (red flour beetle),MVN24565.1,MVN24565.1 tyrosine-type recombinase/integrase...,Bacillota,Staphylococcus aureus,38.2,9.400000e-23,104.0,0.556391,...,"{'Coxiella sp. RIFCSPHIGHO2_12_FULL_42_15', 'C...",set(),set(),{'Burkholderiaceae bacterium'},set(),set(),set(),set(),set(),set()
"GCF_000002335.3;XP_015837071.1;HGT_(178,225)",Bacteria,Tribolium castaneum (red flour beetle),DAP48613.1,DAP48613.1 MAG TPA: transcription termination ...,Uroviricota,Caudoviricetes sp.,58.1,1.770000e-04,46.2,0.895833,...,set(),set(),set(),set(),set(),set(),set(),set(),set(),set()
"GCF_000239435.1;XP_015786976.1;HGT_(26,168)","Archaea,Bacteria",Tetranychus urticae (two-spotted spider mite),MBF2000645.1,MBF2000645.1 fasciclin domain-containing prote...,Cyanobacteriota,Synechococcales cyanobacterium C42_A2020_086;S...,34.4,7.860000e-15,79.3,0.853147,...,set(),set(),set(),set(),set(),set(),set(),set(),set(),set()
"GCF_000239435.1;XP_025018608.1;HGT_(1,93)",Bacteria,Tetranychus urticae (two-spotted spider mite),MBJ7364438.1,MBJ7364438.1 class I tRNA ligase family protei...,Cyanobacteriota,Synechococcus sp. SupBloom_Metag_053,51.0,3.850000e-22,97.4,1.064516,...,"{'Coxiellaceae bacterium', 'Coxiella endosymbi...",set(),set(),{'Burkholderiales bacterium'},set(),set(),set(),set(),set(),set()
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"GCF_917208135.1;XP_048512468.1;HGT_(33,195)",Bacteria,Athalia rosae (coleseed sawfly),MCA0364948.1,MCA0364948.1 MFS transporter [Bacteroidota bac...,Bacteroidota,Bacteroidota bacterium,29.7,2.000000e-07,62.0,0.828221,...,set(),set(),set(),{'Burkholderiales bacterium'},set(),set(),set(),set(),set(),set()
"GCF_921293095.1;XP_046396626.1;HGT_(32,113)",Bacteria,Ischnura elegans (damselflies),WP_238385113.1,WP_238385113.1 DUF3662 and FHA domain-containi...,Actinomycetota,Nesterenkonia muleiensis,33.8,3.470000e-04,49.7,0.939024,...,set(),set(),set(),set(),set(),set(),set(),set(),set(),set()
"GCF_921293095.1;XP_046402901.1;HGT_(329,1335)",Bacteria,Ischnura elegans (damselflies),WP_260536911.1,WP_260536911.1 RHS repeat-associated core doma...,Bacteroidota,Candidatus Cardinium sp. TP,67.7,0.000000e+00,1440.0,1.008937,...,set(),set(),set(),set(),set(),set(),set(),set(),set(),set()
